<a href="https://colab.research.google.com/github/armandossrecife/mysummary/blob/main/extrator_dados_processos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup

def get_html_content(url):
  html_content = ""
  try:
    response = requests.get(url)
    response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

    if response.status_code == 200:
        print("Requisição bem sucedida!")
        html_content = response.content
    else:
        raise ValueError("Requisição falhou com o código de status:", response.status_code)
    return html_content
  except requests.exceptions.RequestException as e:
      raise ValueError("Error fetching the webpage:", e)

def extract_data_from_table(table, headers):
    """Extrai dados de uma tabela HTML e retorna uma lista de dicionários."""
    data = []
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if cells:  # Verifica se a linha contém dados
            row_data = dict(zip(headers, [cell.text.strip() for cell in cells]))
            data.append(row_data)
    return data

class ProcessoExtractor:
    def __init__(self, url):
        self.url = url
        self.soup = self._get_soup()

    def _get_soup(self):
        html_content = get_html_content(self.url)
        return BeautifulSoup(html_content, 'html.parser')

    def extract_interessado(self):
        table = self.soup.find('table', class_='subListagem')
        headers = ['Tipo', 'Identificador', 'Nome']
        return extract_data_from_table(table, headers)

    def extract_documentos(self):
        table = self.soup.find_all('table', class_='subListagem')[1]  # Assumindo que a tabela de documentos é a segunda
        headers = ['Tipo do Documento', 'Data do Documento', 'Origem', 'Natureza']
        return extract_data_from_table(table, headers)

    def extract_movimentacoes(self):
        table = self.soup.find_all('table', class_='subListagem')[2]  # Assumindo que a tabela de movimentações é a terceira
        headers = ['Data Origem', 'Unidade Origem', 'Unidade Destino', 'Enviado Por', 'Recebido Em', 'Recebido Por']
        return extract_data_from_table(table, headers)

    def get_all_data(self):
        return {
            "interessado": self.extract_interessado(),
            "documentos": self.extract_documentos(),
            "movimentacoes": self.extract_movimentacoes()
        }

# Usando a classe
url_processo = "https://www.sipac.ufpi.br/public/jsp/processos/processo_detalhado.jsf?id=666933"
extractor = ProcessoExtractor(url_processo)

# Obtendo todos os dados do processo
dados_processo = extractor.get_all_data()

# Acessando os dados:
print(dados_processo['interessado'])
print(dados_processo['documentos'])
print(dados_processo['movimentacoes'])

Requisição bem sucedida!
[{'Tipo': 'Servidor', 'Identificador': '1886865', 'Nome': 'ARMANDO SOARES SOUSA'}]
[{'Tipo do Documento': 'REQUERIMENTO', 'Data do Documento': '20/08/2024', 'Origem': 'DIPROT/PRAD (11.00.15.08.03)', 'Natureza': 'SIGILOSO'}, {'Tipo do Documento': 'DESPACHO', 'Data do Documento': '21/08/2024', 'Origem': 'CDP/SRH (11.00.15.05.02)', 'Natureza': ''}, {'Tipo do Documento': 'PORTARIA', 'Data do Documento': '12/06/2018', 'Origem': 'DC/CCN (11.00.24.05)', 'Natureza': 'OSTENSIVO'}, {'Tipo do Documento': 'DECLARAÇÃO', 'Data do Documento': '21/08/2024', 'Origem': 'DC/CCN (11.00.24.05)', 'Natureza': 'OSTENSIVO'}, {'Tipo do Documento': 'DESPACHO', 'Data do Documento': '23/08/2024', 'Origem': 'DC/CCN (11.00.24.05)', 'Natureza': 'OSTENSIVO'}, {'Tipo do Documento': 'DESPACHO', 'Data do Documento': '23/08/2024', 'Origem': 'CDP/SRH (11.00.15.05.02)', 'Natureza': ''}, {'Tipo do Documento': 'DESPACHO', 'Data do Documento': '24/09/2024', 'Origem': 'CDP/SRH (11.00.15.05.02)', 'Nature